In [1]:
import init_paths
import argparse
import os
import time
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import math
import numpy as np
from utils import *
from validation import validate
#import torchvision.models as models
import models
from models.imagenet_resnet import BasicBlock, Bottleneck
from multiprocessing import Pool
#from torchvision.models.resnet import BasicBlock, Bottleneck
import pdb
import wandb

from apex import amp
import copy

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

In [2]:
term_width = 192

TOTAL_BAR_LENGTH = 65.
last_time = time.time()
begin_time = last_time

def progress_bar(current, total, msg=None):
    global last_time, begin_time
    if current == 0:
        begin_time = time.time()  # Reset for new bar.

    cur_len = int(TOTAL_BAR_LENGTH*current/total)
    rest_len = int(TOTAL_BAR_LENGTH - cur_len) - 1

    sys.stdout.write(' [')
    for i in range(cur_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    cur_time = time.time()
    step_time = cur_time - last_time
    last_time = cur_time
    tot_time = cur_time - begin_time

    L = []
    L.append('  Step: %s' % format_time(step_time))
    L.append(' | Tot: %s' % format_time(tot_time))
    if msg:
        L.append(' | ' + msg)

    msg = ''.join(L)
    sys.stdout.write(msg)
    for i in range(term_width-int(TOTAL_BAR_LENGTH)-len(msg)-3):
        sys.stdout.write(' ')

    # Go back to the center of the bar.
    for i in range(term_width-int(TOTAL_BAR_LENGTH/2)+2):
        sys.stdout.write('\b')
    sys.stdout.write(' %d/%d ' % (current+1, total))

    if current < total-1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()


def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

In [3]:
# Create the model
arch = 'resnet50'
print("=> creating model '{}'".format(arch))
model = models.__dict__[arch]()
numberofclass = 1000

def init_dist_weights(model):
    for m in model.modules():
        if isinstance(m, BasicBlock):
            m.bn2.weight = nn.Parameter(torch.zeros_like(m.bn2.weight))
        if isinstance(m, Bottleneck):
            m.bn3.weight = nn.Parameter(torch.zeros_like(m.bn3.weight))
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)

init_dist_weights(model)
# Wrap the model into DataParallel
model.cuda()
model = torch.nn.DataParallel(model)

valdir = os.path.join('/workspace/dataset/ILSVRC2012/val')
valdir = os.path.join('/workspace/dataset/ILSVRC2012-sz/352/val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=512, shuffle=False,
    num_workers=10, pin_memory=True)


# # Initiate data loaders
# data = '/workspace/dataset/ILSVRC2012-sz/352'
# crop_size = 224
# min_scale = 0.087
# batch_size = 448
# workers = 10
# traindir = os.path.join(data, 'train')
# valdir = os.path.join(data, 'val')

# train_transform = transforms.Compose([
#     transforms.RandomResizedCrop(crop_size, scale=(min_scale, 1.0)),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor()
# ])

# test_transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
# ])

# train_dataset = datasets.ImageFolder(traindir, train_transform)

# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                            batch_size=batch_size,
#                                            shuffle=True,
#                                            num_workers=workers,
#                                            pin_memory=True,
#                                            sampler=None,
#                                            drop_last=True)

# val_loader = torch.utils.data.DataLoader(datasets.ImageFolder(valdir, test_transform),
#                                          batch_size=batch_size,
#                                          shuffle=False,
#                                          num_workers=workers,
#                                          pin_memory=True,
#                                          drop_last=False)


=> creating model 'resnet50'


In [37]:
resume = ''
if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    checkpoint = torch.load(resume)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(resume,
                                                        checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume))

=> loading checkpoint 'ResNet50_Baseline_23.68'
=> loaded checkpoint 'ResNet50_Baseline_23.68'
DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_

In [5]:
for exp in ['cutmix_d0.9_phase3']:
#             'cutmix_phase3',
#             'vanilla_phase3',
#             'cutmix_d0.0_phase3',
#             'cutmix_d0.1_phase3',
#             'cutmix_d0.3_phase3',
#             'cutmix_d0.5_phase3',
#             'cutmix_d0.6_phase3',
#             'cutmix_d0.7_phase3',
#             'cutmix_d0.8_phase3',
#             'cutmix_d1.0_phase3']:
    top1 = AverageMeter()
    top5 = AverageMeter()
    expname = exp
#     resume = '../../Mixup_imagenet_exp_old/imagenet_fast/trained_models/' + expname + '/model_best.pth.tar'
    resume = 'trained_models/' + expname + '/model_best.pth.tar'
    if os.path.isfile(resume):
        print("=> loading checkpoint '{}'".format(resume))
        checkpoint = torch.load(resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(resume,
                                                            checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resume))

    print('the number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    print('best prec1: {}'.format(best_prec1))

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    cudnn.benchmark = True
    correct_classes = np.zeros(numberofclass, dtype=int)
    number_classes = np.zeros(numberofclass, dtype=int)

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda()

        output = model(input)
        
        prec1, prec5 = accuracy(output, target, topk=(1, 5))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        output = output.data
        topk=(1,)

        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        for c, n in zip(correct.reshape(-1).int(), target):
            correct_classes[n] += c
            number_classes[n] += 1

        progress_bar(i, len(val_loader), 'eval')
    print(' Final Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))
    np.savetxt('recall/' + expname + '_val.csv', np.transpose(np.stack((correct_classes, number_classes, correct_classes/number_classes))), delimiter=',')

=> loading checkpoint 'trained_models/cutmix_d0.9_phase3/model_best.pth.tar'
=> loaded checkpoint 'trained_models/cutmix_d0.9_phase3/model_best.pth.tar' (epoch 93)
the number of model parameters: 25557032
best prec1: 76.2439956665039
 [================================================================>]  Step: 183ms | Tot: 28s972ms | eval                                                                                       98/98 6/98                                                               90/98 
 Final Prec@1 76.256 Prec@5 92.984


In [10]:
for exp in ['cutmix_phase3',
            'vanilla_phase3',
            'cutmix_d0.0_phase3',
            'cutmix_d0.1_phase3',
            'cutmix_d0.3_phase3',
            'cutmix_d0.5_phase3',
            'cutmix_d0.6_phase3',
            'cutmix_d0.7_phase3',
            'cutmix_d0.8_phase3',
            'cutmix_d1.0_phase3']:
    top1 = AverageMeter()
    top5 = AverageMeter()
    expname = exp
#     resume = '../../Mixup_imagenet_exp_old/imagenet_fast/trained_models/' + expname + '/model_best.pth.tar'
    resume = 'trained_models/' + expname + '/model_best.pth.tar'
    if os.path.isfile(resume):
        print("=> loading checkpoint '{}'".format(resume))
        checkpoint = torch.load(resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(resume,
                                                            checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resume))

    print('the number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    print('best prec1: {}'.format(best_prec1))

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    cudnn.benchmark = True
    correct_classes = np.zeros(numberofclass, dtype=int)
    number_classes = np.zeros(numberofclass, dtype=int)

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda()

        output = model(input)
        
        prec1, prec5 = accuracy(output, target, topk=(1, 5))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        output = output.data
        topk=(1,)

        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        for c, n in zip(correct.reshape(-1).int(), target):
            correct_classes[n] += c
            number_classes[n] += 1

        progress_bar(i, len(val_loader), 'eval')
    print(' Final Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))
    np.savetxt('recall/' + expname + '_val.csv', np.transpose(np.stack((correct_classes, number_classes, correct_classes/number_classes))), delimiter=',')

=> loading checkpoint 'trained_models/cutmix_phase3/model_best.pth.tar'
=> loaded checkpoint 'trained_models/cutmix_phase3/model_best.pth.tar' (epoch 90)
the number of model parameters: 25557032
best prec1: 77.09400177001953
 [================================================================>]  Step: 123ms | Tot: 54s792ms | eval                                                                                       98/98 .........................................]  Step: 197ms | Tot: 17s468ms | eval                                                                                       6/98 30/98                                                                38/98 .............................]  Step: 200ms | Tot: 34s413ms | eval                                                                                       40/98 59/98 61/98 62/98                    63/9 72/9 73/98 
 Final Prec@1 76.516 Prec@5 93.206
=> loading checkpoint 'trained_models/vanilla_phase3/model_best.pth.tar'
=> loaded ch

In [61]:
for exp in range(0,11):
    expname = 'cutmix_d' + str(exp/10)
    resume = 'trained_models/' + expname + '_phase1/model_best.pth.tar'
    if os.path.isfile(resume):
        print("=> loading checkpoint '{}'".format(resume))
        checkpoint = torch.load(resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(resume,
                                                            checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resume))

    print('the number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
    print('best prec1: {}'.format(best_prec1))

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    cudnn.benchmark = True
    correct_classes = np.zeros(numberofclass, dtype=int)
    number_classes = np.zeros(numberofclass, dtype=int)

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda()

        output = model(input)
        output = output.data
        topk=(1,)

        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        for c, n in zip(correct.reshape(-1).int(), target):
            correct_classes[n] += c
            number_classes[n] += 1

        progress_bar(i, len(val_loader), 'eval')
    np.savetxt('recall/' + expname + '_val.csv', np.transpose(np.stack((correct_classes, number_classes, correct_classes/number_classes))), delimiter=',')

=> loading checkpoint 'trained_models/cutmix_d0.0_phase1/model_best.pth.tar'
=> loaded checkpoint 'trained_models/cutmix_d0.0_phase1/model_best.pth.tar' (epoch 100)
the number of model parameters: 25557032
best prec1: 77.19400024414062
 [================================================================>]  Step: 120ms | Tot: 36s842ms | eval                                                                                       98/98 98 9/9 18/98 .....]  Step: 193ms | Tot: 18s27ms | eval                                                                                        39/98                                  40/9 62/98                                                                       69/98 
=> loading checkpoint 'trained_models/cutmix_d0.1_phase1/model_best.pth.tar'
=> loaded checkpoint 'trained_models/cutmix_d0.1_phase1/model_best.pth.tar' (epoch 94)
the number of model parameters: 25557032
best prec1: 76.9964599609375
 [==============================================================

In [15]:
output.shape, target.shape

(torch.Size([256, 1000]), torch.Size([256]))

In [14]:
import numpy as np
numberofclass = 1000
correct_classes = torch.empty(numberofclass, 2)

In [19]:
correct_classes[500][1]

tensor(1.8177e+31)

In [6]:
degraded_cls_list = torch.tensor([266, 193, 250, 660, 764, 620, 380, 409, 676, 887, 928, 975, 171, 311, 377, 857, 101, 46, 543, 559, 579, 803, 813, 885, 32, 748, 63, 142, 161, 186, 402, 629, 868, 878, 908, 233, 249, 304, 489, 506, 600, 696, 812, 814, 911, 923, 26, 77, 134, 202, 210, 243, 328, 382, 385, 473, 486, 522, 667, 711, 795, 797, 830, 902, 950, 68, 282, 415, 664, 838, 841, 860, 978, 999, 50, 184, 197, 383, 412, 457, 500, 507, 650, 693, 759, 778, 825, 844, 931, 939, 43, 81, 140, 165, 252, 297, 331, 345, 367, 379, 468, 546, 645, 726, 754, 756, 859, 864, 901, 972, 983, 4, 25, 115, 126, 152, 163, 204, 258, 259, 322, 442, 487, 538, 585, 589, 593, 596, 606, 617, 622, 710, 852, 914, 958, 990, 60, 73, 493, 567, 657, 848, 240, 44, 45, 71, 94, 158, 159, 178, 188, 199, 217, 288, 337, 349, 438, 466, 535, 578, 648, 678, 694, 735, 780, 786, 808, 880, 884, 980, 9, 90, 139, 182, 223, 230, 287, 295, 313, 356, 375, 448, 513, 708, 736, 740, 793, 799, 826, 856, 867, 889, 917, 918, 943, 7, 18, 23, 24, 34, 69, 74, 75, 78, 80, 82, 89, 92, 96, 97, 103, 116, 122, 131, 144, 157, 168, 174, 185, 194, 195, 213, 231, 234, 248, 261, 262, 272, 273, 274, 279, 284, 302, 307, 317, 338, 342, 343, 352, 354, 355, 368, 378, 381, 399, 420, 431, 434, 436, 437, 447, 452, 456, 460, 461, 480, 483, 503, 511, 523, 529, 530, 545, 582, 586, 626, 644, 653, 654, 692, 766, 784, 790, 800, 805, 817, 823, 827, 849, 863, 876, 892, 906, 907, 921, 936, 938, 942, 949, 960, 984, 987, 988, 992, 836, 499, 633, 88, 99, 138, 143, 181, 226, 236, 253, 283, 286, 306, 316, 318, 323, 599, 625, 671, 730, 731, 779, 816, 953])

In [8]:
degraded_cls_list.shape

torch.Size([319])

In [9]:
degraded_cls = degraded_cls_list[0:50]

In [11]:
degraded_cls

tensor([266, 193, 250, 660, 764, 620, 380, 409, 676, 887, 928, 975, 171, 311,
        377, 857, 101,  46, 543, 559, 579, 803, 813, 885,  32, 748,  63, 142,
        161, 186, 402, 629, 868, 878, 908, 233, 249, 304, 489, 506, 600, 696,
        812, 814, 911, 923,  26,  77, 134, 202])

In [13]:
torch.zeros(len(degraded_cls)).bool()

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])